In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.flow import edmonds_karp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Wrangling

In order to build a proper flow network, we need 3 things:
- Vertices (airports)
- Edges (routes)
- Capacities of edges (# of seats on route)

The data needed to construct the vertices and edges are supplied in the files read below; however, the *capacities* will need to be assigned to the routed using the data provided in these files.

Since these capacities are determined by the number of seats that can be booked on a given *route*, we will have to find a way to pair the **Equipment** data in the last column of the `routesDF` with the three digit IATA codes in the 2nd column of the `planesDF`.

In [ ]:
pathToFile = "/content/drive/MyDrive/AlgoProj2_Files"

zDF = pd.read_csv(f"{pathToFile}/routes.dat.txt", header=None  )
planesDF = pd.read_csv(f"{pathToFile}/planes.dat.txt", header = None)
planesDF.columns = ["Airplane","IATA","ICAO"]
routesDF = pd.read_csv(f"{pathToFile}/routes.dat.txt", header = None)
routesDF.columns = ["Airline Code", "AirlineID", "Source", "SourceID", "Destination", "DestinationID", "cShare", "NumStops", "Equipment"]

In [ ]:
planesDF.shape
planesDF.head()

,Airplane,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [ ]:
routesDF.shape
routesDF.head()

,Airline Code,AirlineID,Source,SourceID,Destination,DestinationID,cShare,NumStops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


## Removing NAs

Prior to associating capacities, it will be beneficial to reduce our `planesDF` to a dataframe that contains only the information relevant to our `routesDF` data.

To do this, we extract the rows from `planesDF` that have IATA codes in the Equipment column of the `routesDF`. In addition to reducing our data to only those values relevant to our project, this code will also remove any NAs (encoded as `\N`) from our dataset.  

In [ ]:
planesDF[planesDF['IATA'].isin(routesDF['Equipment'])].shape

(126, 3)

## Reducing Multiple

Certain rows in the routesDF data contain multiple entries for equipment, separated by spaces. Since we want one capacity for each route, we need a way of reducing the values into one, which represents the capacity of this connection.

There are three ways of approaching this:
- Min
  - Use the smallest capacity for the route
- Avg
  - Use the average of equipment capacities
- Max
  - Use the largest capacity for the route

## Attaching Capacities

Unsure of how to derive it from the data set supplied with the assignment, we set out to find supplementary datasets that may contain the information we need.

http://www.lsv.fr/~sirangel/teaching/dataset/aircrafts.txt

In [ ]:
aircraftsDF = pd.read_csv(f"{pathToFile}/aircrafts.txt", header = None, delimiter= ';')

In [ ]:
aircraftsDF.columns = ["Airplane","ICAO","Equipment", "Capacity", "Country"]
aircraftsDF.head()

,Airplane,ICAO,Equipment,Capacity,Country
0,Aerospatiale/Alenia ATR 42-300 / 320,AT43,AT4,50,France
1,Aerospatiale/Alenia ATR 42-500,AT45,AT5,50,France
2,Aerospatiale/Alenia ATR 42/ ATR 72,\N,ATR,74,France
3,Aerospatiale/Alenia ATR 72,AT72,AT7,74,France
4,Aerospatiale/BAC Concorde,CONC,SSC,128,France


Next, we need attach the seating capacity from `aircraftsDF` to the `routesDF` data using `pd.merge()`

In [ ]:
capDict = {'A81' : 85,  'AN4' : 44, 'BNI' : 9, 'CNC' : 19, 
        'DHP' : 6, 'DHT' : 20, 'BET' : 1,  '73M' :161.5, 
        'SU9' : 1, 'PL2' :9, 'PAG' :9, 'YK2' :120, 'YK4' :24,  
        'PA2' : 9, 'CN1' : 1, 'CNA' : 1, 'BE1'  :19, 'CNT' : 14, 
        'MA6' : 1, '77W' : 396, '772' : 368,  'CRK' : 100, '787' : 310, 
        '32B' : 244,  '32A' : 150,  '73C' : 1, 'CRA' : 1, '77L' : 317, 
        '788' : 242,  '76W'  : 1, '74Y' : 1, '74H' :467, '73J' : 1, 
        '73Q' : 1, '75W' : 1, 'F28' : 65, '74N' : 1, 'YN7' : 52, 
        'IL9' : 262, 'A58' : 75, '75T' : 1,  'BH2' : 14, 'NDE' : 5, 
        'BEC' : 2, 'CNJ' : 10,  'J32' : 19, 'AB4' : 247, 'PA1' : 9,  
        'BE9' : 16, 'M1F' : 1, 'YN2' : 17, '76F' : 1,  'CN2' : 19, 'SFB' : 1, 
        '73R' : 150, '73N' : 149,  '77X' : 1, '33X' : 1,  '32C': 107}

for index, row in aircraftsDF.iterrows():
  if row['Equipment'] != '\\N' and row['Capacity'] != '\\N':
    capDict[row['Equipment']] = int(row['Capacity'])

routesDF['MinCapacity'] = np.nan
routesDF['AvgCapacity'] = np.nan
routesDF['MaxCapacity'] = np.nan

for i in range(len(routesDF.Equipment)):
  planes = str(routesDF.Equipment[i]).split(' ')
  capacities = []
  for plane in planes:
    if plane != '' and plane != 'nan' and plane != '\\N':
      capacities.append(capDict[plane])
  if capacities != []:
    routesDF['MinCapacity'][i] = min(capacities)
    routesDF['AvgCapacity'][i] = round(sum(capacities)/len(capacities))
    routesDF['MaxCapacity'][i] = max(capacities)

routesDF = routesDF.dropna(subset=['MinCapacity','AvgCapacity','MaxCapacity'])
routesDF = routesDF[routesDF.Source != '\\N']
routesDF = routesDF[routesDF.Destination != '\\N']

routesDF.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Airline Code,AirlineID,Source,SourceID,Destination,DestinationID,cShare,NumStops,Equipment,MinCapacity,AvgCapacity,MaxCapacity
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,50.0,50.0,50.0
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2,50.0,50.0,50.0
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2,50.0,50.0,50.0
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2,50.0,50.0,50.0
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2,50.0,50.0,50.0


## Super Node ID Columns

In [ ]:
airportsDF = pd.read_csv(f"{pathToFile}/airports.dat.txt", header=None  )
apcodesDF = airportsDF[[2,3,4]]
airportsDF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


### Source

In [ ]:
sourcecodesDF = apcodesDF
sourcecodesDF.columns = ["SourceCity", "SourceCountry", "Source"]

srcDF = pd.merge(routesDF, sourcecodesDF, how="inner", on = "Source")
srcDF

,Airline Code,AirlineID,Source,SourceID,Destination,DestinationID,cShare,NumStops,Equipment,MinCapacity,AvgCapacity,MaxCapacity,SourceCity,SourceCountry
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,50.0,50.0,50.0,Sochi,Russia
1,7J,9531,AER,2965,DYU,2979,NaN,0,735,108.0,108.0,108.0,Sochi,Russia
2,9U,1073,AER,2965,KIV,1735,NaN,0,EM2,30.0,30.0,30.0,Sochi,Russia
3,B2,1478,AER,2965,MSQ,2954,NaN,0,735 CRJ,70.0,89.0,108.0,Sochi,Russia
4,HY,5281,AER,2965,TAS,2983,NaN,0,767,216.0,216.0,216.0,Sochi,Russia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67234,ZL,4178,TRO,6794,GFN,6792,NaN,0,SF3,37.0,37.0,37.0,Taree,Australia
67235,ZL,4178,TRO,6794,SYD,3361,NaN,0,SF3,37.0,37.0,37.0,Taree,Australia
67236,ZL,4178,WIN,6337,LRE,6289,NaN,0,SF3,37.0,37.0,37.0,Winton,Australia
67237,ZL,4178,WIN,6337,TSV,3330,NaN,0,SF3,37.0,37.0,37.0,Winton,Australia


### Destination

In [ ]:
destcodesDF = apcodesDF
destcodesDF.columns = ["DestCity", "DestCountry", "Destination"]

routesDF2 = pd.merge(srcDF, destcodesDF, how="inner", on = "Destination")
routesDF2

,Airline Code,AirlineID,Source,SourceID,Destination,DestinationID,cShare,NumStops,Equipment,MinCapacity,AvgCapacity,MaxCapacity,SourceCity,SourceCountry,DestCity,DestCountry
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,50.0,50.0,50.0,Sochi,Russia,Kazan,Russia
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2,50.0,50.0,50.0,Astrakhan,Russia,Kazan,Russia
2,2B,410,CEK,2968,KZN,2990,NaN,0,CR2,50.0,50.0,50.0,Chelyabinsk,Russia,Kazan,Russia
3,2B,410,DME,4029,KZN,2990,NaN,0,CR2,50.0,50.0,50.0,Moscow,Russia,Kazan,Russia
4,S7,4329,DME,4029,KZN,2990,NaN,0,319,124.0,124.0,124.0,Moscow,Russia,Kazan,Russia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66911,YN,\N,ZKE,5543,YFA,5490,NaN,0,DH1,40.0,40.0,40.0,Kashechewan,Canada,Fort Albany,Canada
66912,YN,\N,YPO,5522,YAT,5482,NaN,0,DH1,40.0,40.0,40.0,Peawanuck,Canada,Attawapiskat,Canada
66913,YN,\N,ZKE,5543,YAT,5482,NaN,0,DH1,40.0,40.0,40.0,Kashechewan,Canada,Attawapiskat,Canada
66914,ZL,4178,JCK,6276,RCM,9904,NaN,0,SF3,37.0,37.0,37.0,Julia Creek,Australia,Richmond,Australia


### Airport Dictionary



In [ ]:
srcs = routesDF2.filter(regex = ("Source*")).drop_duplicates()

In [ ]:
srcsDict = {}

for i in srcs['SourceCity'].unique():
    srcsDict[i] = [j for j in srcs[srcs['SourceCity']==i].Source]

srcsDict['New York']

['JFK', 'LGA']

In [ ]:
dest = routesDF2.filter(regex = ("Dest*")).drop_duplicates()

In [ ]:
destDict = {}

for i in dest['DestCity'].unique():
    destDict[i] = [j for j in dest[dest['DestCity']==i].Destination]

In [ ]:
destDict["San Francisco"]

['SFO']

# Graph Class & Ford-Fulkerson
The following classes allow creation of vertices and edges in a directed graph. The getPath and calculateMaxFlow functions (under the FlowNetwork class) can be used to find max flow from source to sink.
These classes will be useful in finding the max number of people we can transport from one airport to another.

In [ ]:
class Graph: 
   
    def __init__(self,graph): 
        self.graph = graph # residual graph 
        self. ROW = len(graph) 
        #self.COL = len(gr[0]) 
          
   
    '''Returns true if there is a path from source 's' to sink 't' in 
    residual graph. Also fills parent[] to store the path '''
    def BFS(self,s, t, parent): 
  
        # Mark all the vertices as not visited 
        visited =[False]*(self.ROW) 
          
        # Create a queue for BFS 
        queue=[] 
          
        # Mark the source node as visited and enqueue it 
        queue.append(s) 
        visited[s] = True
           
         # Standard BFS Loop 
        while queue: 
  
            #Dequeue a vertex from queue and print it 
            u = queue.pop(0) 
          
            # Get all adjacent vertices of the dequeued vertex u 
            # If a adjacent has not been visited, then mark it 
            # visited and enqueue it 
            for ind, val in enumerate(self.graph[u]): 
                if visited[ind] == False and val > 0 : 
                    queue.append(ind) 
                    visited[ind] = True
                    parent[ind] = u 
  
        # If we reached sink in BFS starting from source, then return 
        # true, else false 
        return True if visited[t] else False
              
      
    # Returns tne maximum flow from s to t in the given graph 
    def FordFulkerson(self, source, sink): 
  
        # This array is filled by BFS and to store path 
        parent = [-1]*(self.ROW) 
  
        max_flow = 0 # There is no flow initially 
  
        # Augment the flow while there is path from source to sink 
        while self.BFS(source, sink, parent) : 
  
            # Find minimum residual capacity of the edges along the 
            # path filled by BFS. Or we can say find the maximum flow 
            # through the path found. 
            path_flow = float("Inf") 
            s = sink 
            while(s !=  source): 
                path_flow = min (path_flow, self.graph[parent[s]][s]) 
                s = parent[s] 
  
            # Add path flow to overall flow 
            max_flow +=  path_flow 
  
            # update residual capacities of the edges and reverse edges 
            # along the path 
            v = sink 
            while(v !=  source): 
                u = parent[v] 
                self.graph[u][v] -= path_flow 
                self.graph[v][u] += path_flow 
                v = parent[v] 
  
        return max_flow 

# Max Flow Function/Creating the Network

To create our network, we first need to establish the columns where we will find our vertices, with which we can identify the directed edges) and the capacities of their connections.
- Vertices (Edges)
  - Source Airport = `graphDF["Source"]`
  - Destination Airport = `graphDF["Destination"]`
- Edges
  - Plane Capacity = `graphDF["MinCapacity" | "MaxCapacity" | "AvgCapacity"]`

In [ ]:
def maxFlow(routesDF, airlineID=None, sources=['JFK','LGA','EWR'], sinks=['SFO','SJC','OAK','STS'], capEstimate='AvgCapacity'):

    if airlineID != None:
      routesDF = routesDF[routesDF.AirlineID == airlineID]
    
    newDF = routesDF[(routesDF['NumStops'] != 1) | ((routesDF['Source'].isin(sources)) & (routesDF['Destination'].isin(sinks)))]

    graphDF = newDF.groupby(['Source','Destination'],as_index=False).agg({'MinCapacity': 'sum', 'AvgCapacity': 'sum','MaxCapacity':'sum'})
    
    graphDF = graphDF[(~graphDF.Destination.isin(sources)) & (~graphDF.Source.isin(sinks)) & ((graphDF.Destination.isin(sinks))|(graphDF.Source.isin(sources)))]

    newDF = routesDF[(routesDF['NumStops'] != 1) | ((routesDF['Source'].isin(sources)) & (routesDF['Destination'].isin(sinks)))]

    graphDF = newDF.groupby(['Source','Destination'],as_index=False).agg({'MinCapacity': 'sum', 'AvgCapacity': 'sum','MaxCapacity':'sum'})

    graphDF = graphDF[(~graphDF.Destination.isin(sources)) & (~graphDF.Source.isin(sinks)) & ((graphDF.Destination.isin(sinks))|(graphDF.Source.isin(sources)))]

    #Create empty list to store Edges
    edgeList = [] 
    #Iterate through DataFrame, appending each edge to new list
    for index, row in graphDF.iterrows(): 
        edgeList.append([row['Source'], row['Destination'], int(row[capEstimate])])

    for airport in sources:
      edgeList.append(['s',airport,np.inf])
    for airport in sinks:
      edgeList.append([airport,'t',np.inf])

    airports = []
    for i in edgeList:
      for j in i:
        if (type(j) == str) and j not in airports:
          airports.append(j)
    numbers = list(range(len(airports)))
    airportDict = dict(zip(airports, numbers))

    numEdgeList = []
    for i in edgeList:
        edge = []
        for j in i:
            if type(j) == str:
                edge.append(airportDict[j])
            else:
                edge.append(j)
        numEdgeList.append(edge)

    A = np.zeros((len(airportDict),len(airportDict)), dtype =float)
    for i in numEdgeList:
        A[i[0]][i[1]] = i[2]

    G = Graph(A)
    source = airportDict['s']
    sink = airportDict['t']

    return G.FordFulkerson(source, sink)



This code chunk gives us the number of people that can be transferred from JFK to SFO (using the average plane capacity per flight). 33372 people can be transported.

In [ ]:
routesDF[routesDF.Source=='SFO'] # Find Source for San Francisco Airport
routesDF[routesDF.Source=='JFK'] # Find Source for John F. Kennedy (NY) Airport

# Starting airport JFK and ending in SFO
source = ['JFK']
sink = ['SFO']

maxFlow(routesDF, sources = source, sinks = sink, capEstimate='MaxCapacity') 
# 33372 people, using average plane capacity for each row, JFK->SFO

33372.0

This code chunk gives us the maximum number of people we can transport using all of the major airports in NY metropolitan area and SF Bay area (and our max plane capacity estimate): 48875.

In [ ]:
sources=['JFK','LGA','EWR'] # Major airports in NY Metropolitan area
sinks=['SFO','OAK','STS','SJC'] # Major airports in SF Bay area

maxFlow(routesDF, sources=sources, sinks=sinks, capEstimate='MaxCapacity') 
# 48875 people using major airports and largest available plane for each route

48875.0

# Most Effective Carrier From NY Metropolitan to SF Bay Area

The following code gives us a dataframe of how many people can be transported from NY to SF, using the major airports in each city, for each carrier.

In [ ]:
carriersDF = pd.DataFrame(columns=['AirlineCode','AirlineID','MaxPeopleLowEst',
                                   'MaxPeopleAvgEst','MaxPeopleHighEst'])

for i in list(routesDF.AirlineID.unique()):

  maxPeopleHighEst = maxFlow(routesDF,i,capEstimate='MaxCapacity')
  maxPeopleLowEst = maxFlow(routesDF,i,capEstimate='MinCapacity')
  maxPeopleAvgEst = maxFlow(routesDF,i,capEstimate='AvgCapacity')
  carrier = i
  try:
    airlineCode = list(routesDF[routesDF['AirlineID']==i]['Airline Code'])[0]
  except KeyError:
    airlineCode = None

  carriersDF = carriersDF.append({'AirlineCode':airlineCode,
                                  'AirlineID' : carrier ,
                                  'MaxPeopleLowEst' : maxPeopleLowEst,
                                  'MaxPeopleAvgEst' : maxPeopleAvgEst,
                                  'MaxPeopleHighEst' : maxPeopleHighEst} ,
                                  ignore_index=True)

carriersDF.head()

,AirlineCode,AirlineID,MaxPeopleLowEst,MaxPeopleAvgEst,MaxPeopleHighEst
0,2B,410,0,0,0
1,2G,1654,0,0,0
2,2I,8359,0,0,0
3,2J,470,0,0,0
4,2K,1338,0,0,0


We can then determine which airline carrier can transport the most people from NY to SF using the carriersDF. The following code shows us that United Airlines can transport the most people. Assuming we use the largest available plane on each flight, they can transport 10250 people from NY to SF.

In [ ]:
numPeople = max(list(carriersDF.MaxPeopleHighEst)) # The most effective carrier can transport 10250 people from NY to SF
ID = list(carriersDF[carriersDF.MaxPeopleHighEst == numPeople].AirlineID)[0] # The carrierID (AirlineID) is '5209'
list(routesDF[routesDF.AirlineID=='5209']['Airline Code'])[0] # Most effective carrier is United Airlines

'UA'

In [ ]:
carriersDF = carriersDF.sort_values(by=['MaxPeopleHighEst'],ascending=False)
carriersDF.head(20)

,AirlineCode,AirlineID,MaxPeopleLowEst,MaxPeopleAvgEst,MaxPeopleHighEst
464,UA,5209,8137,9251,10250
153,DL,2009,2939,3371,3870
89,AA,24,3062,3226,3433
475,US,5265,2460,2601,2779
110,B6,3029,1492,1518,1544
292,LH,3320,966,1109,1232
507,WN,4547,1134,1152,1170
278,KL,3090,606,691,847
95,AF,137,471,597,824
495,VX,5331,698,724,750


# maxFlowCity
### A function to find the max number of people that can be transported by user's start/end city input

In [ ]:
def maxFlowCity(routesDF, airlineID=None, startCity='New York', endCity='San Francisco', capEstimate='AvgCapacity'):
    
    sources = srcsDict[startCity]
    sinks = srcsDict[endCity]

    if airlineID != None:
      routesDF = routesDF[routesDF.AirlineID == airlineID]
    
    newDF = routesDF[(routesDF['NumStops'] != 1) | ((routesDF['Source'].isin(sources)) & (routesDF['Destination'].isin(sinks)))]

    graphDF = newDF.groupby(['Source','Destination'],as_index=False).agg({'MinCapacity': 'sum', 'AvgCapacity': 'sum','MaxCapacity':'sum'})
    
    graphDF = graphDF[(~graphDF.Destination.isin(sources)) & (~graphDF.Source.isin(sinks)) & ((graphDF.Destination.isin(sinks))|(graphDF.Source.isin(sources)))]

    newDF = routesDF[(routesDF['NumStops'] != 1) | ((routesDF['Source'].isin(sources)) & (routesDF['Destination'].isin(sinks)))]

    graphDF = newDF.groupby(['Source','Destination'],as_index=False).agg({'MinCapacity': 'sum', 'AvgCapacity': 'sum','MaxCapacity':'sum'})

    graphDF = graphDF[(~graphDF.Destination.isin(sources)) & (~graphDF.Source.isin(sinks)) & ((graphDF.Destination.isin(sinks))|(graphDF.Source.isin(sources)))]

    edgeList = []
    for index, row in graphDF.iterrows():
        edgeList.append([row['Source'], row['Destination'], int(row[capEstimate])])

    for airport in sources:
      edgeList.append(['s',airport,np.inf])
    for airport in sinks:
      edgeList.append([airport,'t',np.inf])

    airports = []
    for i in edgeList:
      for j in i:
        if (type(j) == str) and j not in airports:
          airports.append(j)
    numbers = list(range(len(airports)))
    airportDict = dict(zip(airports, numbers))

    numEdgeList = []
    for i in edgeList:
        edge = []
        for j in i:
            if type(j) == str:
                edge.append(airportDict[j])
            else:
                edge.append(j)
        numEdgeList.append(edge)

    A = np.zeros((len(airportDict),len(airportDict)), dtype =float)
    for i in numEdgeList:
        A[i[0]][i[1]] = i[2]

    G = Graph(A)
    source = airportDict['s']
    sink = airportDict['t']

    return G.FordFulkerson(source, sink)



### Example of start-city Tampa and end-city Burlington
1062 people can be transported using our max-plane-capacity-per-route estimate, and all airline carriers.


In [ ]:
maxFlowCity(routesDF,startCity='Tampa',endCity='Burlington',capEstimate='MaxCapacity')

1062.0

## Estimates Using City Labels

We can use maxFlowCity to calculate estimates of the number of people we can transport to SF from NY, using the airports listed in those cities (as opposed to our previous solution, which used airports in the NY metropolitan area and SF Bay Area). 36805 people can be transported using this model.

In [ ]:
maxFlowCity(routesDF, startCity='New York', endCity='San Francisco', capEstimate='MaxCapacity') 

36805.0

### Most Effective Airline Using City Labels

In [ ]:
carriersDF = pd.DataFrame(columns=['AirlineCode','AirlineID','MaxPeopleLowEst',
                                   'MaxPeopleAvgEst','MaxPeopleHighEst'])

for i in list(routesDF.AirlineID.unique()):

  maxPeopleHighEst = maxFlowCity(routesDF,i,startCity='New York',
                                 endCity='San Francisco',capEstimate='MaxCapacity')
  maxPeopleLowEst = maxFlowCity(routesDF,i,startCity='New York',
                                endCity='San Francisco',capEstimate='MinCapacity')
  maxPeopleAvgEst = maxFlowCity(routesDF,i,startCity='New York',
                                endCity='San Francisco',capEstimate='AvgCapacity')
  carrier = i
  try:
    airlineCode = list(routesDF[routesDF['AirlineID']==i]['Airline Code'])[0]
  except KeyError:
    airlineCode = None

  carriersDF = carriersDF.append({'AirlineCode':airlineCode, 'AirlineID' : carrier ,
                                  'MaxPeopleLowEst' : maxPeopleLowEst,
                                  'MaxPeopleAvgEst' : maxPeopleAvgEst,
                                  'MaxPeopleHighEst' : maxPeopleHighEst} ,
                                  ignore_index=True)

carriersDF.head()

,AirlineCode,AirlineID,MaxPeopleLowEst,MaxPeopleAvgEst,MaxPeopleHighEst
0,2B,410,0,0,0
1,2G,1654,0,0,0
2,2I,8359,0,0,0
3,2J,470,0,0,0
4,2K,1338,0,0,0


We can see that United Airlines is still the most effective at transporting people from NY to SF, even when we confine the starting and ending airports to just ones listed in those cities.

In [ ]:
carriersDF = carriersDF.sort_values(by=['MaxPeopleHighEst'],ascending=False)
carriersDF.head(20)

,AirlineCode,AirlineID,MaxPeopleLowEst,MaxPeopleAvgEst,MaxPeopleHighEst
464,UA,5209,3299,3530,3674
153,DL,2009,2118,2570,3096
89,AA,24,2095,2293,2535
475,US,5265,1836,1974,2163
110,B6,3029,1094,1120,1146
292,LH,3320,751,821,890
95,AF,137,470,596,824
278,KL,3090,443,528,685
492,VS,5347,459,537,594
102,AS,439,349,512,593


### Validation Flow Algorithm Works

In [ ]:
graph = [[0, 16, 13, 0, 0, 0], 
        [0, 0, 10, 12, 0, 0], 
        [0, 4, 0, 0, 14, 0], 
        [0, 0, 9, 0, 0, 20], 
        [0, 0, 0, 7, 0, 4], 
        [0, 0, 0, 0, 0, 0]] 
  
g = Graph(graph) 
  
source = 0; sink = 5
   
print ("The maximum possible flow is %d " % g.FordFulkerson(source, sink)) 

The maximum possible flow is 23 


In [ ]:
G = nx.DiGraph()

newDF = routesDF[(routesDF['NumStops'] != 1) | ((routesDF['SourceID']=='3797') & (routesDF['DestinationID']=='3469'))]

graphDF = newDF.groupby(['SourceID','DestinationID'],as_index=False).agg({'MinCapacity': 'sum', 'AvgCapacity': 'sum','MaxCapacity':'sum'})

# Starting airport JFK and ending in SFO
for i in range(len(graphDF.index)):
  if graphDF.DestinationID[i]!='3797' and graphDF.SourceID[i]!='3469' and (graphDF.DestinationID[i]=='3469' or graphDF.SourceID[i]=='3797'):
    G.add_edge(graphDF.SourceID[i], graphDF.DestinationID[i], capacity=graphDF.AvgCapacity[i])

nx.maximum_flow_value(G, "3797", "3469")

31432.0